In [1]:
import numpy as np
import random
import os
import copy

from collections import deque
import gym

In [2]:
import tensorflow as tf

from tensorflow.keras.models import Sequential, clone_model
from tensorflow.keras.layers import Dense, Activation, LeakyReLU, BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam

os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private'
os.environ['TF_GPU_THREAD_COUNT'] = '4' #if not hvd_utils.is_using_hvd() else str(hvd.size())

from rl.agents.dqn import DQNAgent

2022-09-25 18:33:50.528335: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
env_name = "othello:othello-v0"
env = gym.make(env_name)
env.observation_space['state']

Box(0.0, 64.0, (64,), float32)

In [4]:
# import othello
from othello import othello_agent

import importlib
import sys
importlib.reload(sys.modules.get('othello.othello_agent'))


agent = othello_agent.OthelloDQN(nb_observations=64, player="white")
agent.model_target.summary()
# agent.model_eval.summary()

Instructions for updating:
Colocations handled automatically by placer.


/Users/stb/miniforge3/envs/tfrl-metal_py39/lib/python3.9/site-packages/keras/backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 1, 16)             1040      
                                                                 
 dense_9 (Dense)             (None, 1, 32)             544       
                                                                 
 dense_10 (Dense)            (None, 1, 64)             2112      
                                                                 
 batch_normalization_2 (Batc  (None, 1, 64)            256       
 hNormalization)                                                 
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 1, 64)             0         
                                                                 
 dense_11 (Dense)            (None, 1, 128)            8320      
                                                      

In [5]:
# from tensorflow.keras.backend import eval 

# obs, info = env.reset()
# next_player = info["next_player"]["name"]
# next_possible_actions = info["next_possible_actions"]

# action = agent.choose_action(obs["state"].reshape((1,64)), next_possible_actions)
# # action = int(eval(action))
# y_ind = action % 8
# x_ind = (action // 8) % 8

# print(action, x_ind, y_ind)
# assert (x_ind, y_ind) in env.next_possible_actions, "Invalid Next Action"

In [6]:
max_epoch = 20000
best_so_far = 0

for epoch in range(1):
    ep_reward = []
    obs, info = env.reset()
    done = False
    
    
    while True:
        next_player = info["next_player"]["name"]
        next_possible_actions = info["next_possible_actions"]

        if next_player == "white":  # We train the white
            action = agent.choose_action(obs["state"].reshape(1, 64), next_possible_actions)
            obs_, reward, done, _, info = env.step(action)
            ep_reward.append(reward)
            agent.store_transition(obs["state"], action, reward, done, obs_["state"], next_possible_actions)
        else:
            action = random.choice(list(next_possible_actions))
            action = (action[0] * 8) + action[1]
            obs_, reward, done, _, info = env.step(action)

            if done:
                if info["winner"] == "Black":  # when black take the last turn and game over, rewards of white player should be updated
                    agent.reward_transition_update(-10.)
                elif info["winner"] == "White":
                    agent.reward_transition_update(10.)
                else:  # "Tie"
                    agent.reward_transition_update(2.)
        
        obs = copy.deepcopy(obs_)
        
        if agent.memory_counter == 10:
            loss = agent.learn()

        
        
        if done:  # Game Over
            loss = agent.learn()
            print("ep: {:d}/{:d}, white player taining loss value: {:.4f}".format(ep, max_epoch, loss))

/Users/stb/miniforge3/envs/tfrl-metal_py39/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:141: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/Users/stb/miniforge3/envs/tfrl-metal_py39/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/Users/stb/miniforge3/envs/tfrl-metal_py39/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:238: UserWarning: WARN: Expects `truncated` signal to be a boolean, actual type: <class 'int'>
  logger.warn(
/Users/stb/miniforge3/envs/tfrl-metal_py39/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:141: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/Users/stb/minif

Metal device set to: AMD Radeon Pro 555


2022-09-25 18:34:00.512131: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-09-25 18:34:00.540936: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-09-25 18:34:00.615558: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-09-25 18:34:00.742535: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-09-25 18:34:01.159392: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-09-25 18:34:01.367746: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-09-25 18:34:01.736495: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_reg

(32, 1, 64) (32, 1, 64)


2022-09-25 18:34:07.202963: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-09-25 18:34:07.286248: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-09-25 18:34:07.339934: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-09-25 18:34:07.407040: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


InvalidArgumentError: 2 root error(s) found.
  (0) INVALID_ARGUMENT: Can not squeeze dim[2], expected a dimension of 1, got 64
	 [[{{node metrics/accuracy/Squeeze}}]]
	 [[dense_3/Tensordot/Prod_1/_1481]]
  (1) INVALID_ARGUMENT: Can not squeeze dim[2], expected a dimension of 1, got 64
	 [[{{node metrics/accuracy/Squeeze}}]]
0 successful operations.
0 derived errors ignored.

In [ ]:
print(obs, next_player, next_possible_actions)

In [ ]:
info["next_player"]["name"]
